In [1]:
!pip install matplotlib
!pip install seaborn 
!pip install numpy 
!pip install rdata
!pip install pyreadr

In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels.api as sm
from pathlib import Path
import os
import pickle
import rdata
import pyreadr

In [3]:
#Read in all data
data = pd.read_csv("clim_conflict_for_R.csv")
area = pd.read_csv("croparea_weight.csv", encoding="windows-1252")
ssa = pd.read_csv("gseasSSA.csv")
cltol = pd.read_csv("precip_climatology.csv", index_col=0)
mss_country = pd.read_csv("mss_countries_new.csv", header=0)
mss_country.iloc[:, 0] = mss_country.iloc[:, 0].astype(str)
mss = mss_country.iloc[:, 0].unique().tolist()

In [4]:
# Define the hold list with specified column combinations

hold = []

# Helper to select columns by index
def select_columns(indices):
    return data.iloc[:, indices].copy()

#Create dataframes for each model 
hold.append(select_columns([105, 41, 44] + list(range(109, 150)) + list(range(177, 218))))  # Model 1
hold.append(select_columns([105, 41, 44, 53, 56] + list(range(109, 150)) + list(range(177, 218))))  # Model 2
hold.append(select_columns([105, 41, 44, 9, 227] + list(range(109, 150)) + [176]))  # Model 3
hold.append(select_columns([105, 41, 44, 53, 56, 9, 227] + list(range(109, 150)) + [176]))  # Model 4
hold.append(select_columns([105, 41, 44, 53, 56] + list(range(109, 150)) + [176]))  # Model 5
hold.append(select_columns([105, 47, 50, 59, 62] + list(range(109, 150)) + list(range(177, 218))))  # Model 6
hold.append(select_columns([105] + list(range(218, 222)) + list(range(109, 150)) + list(range(177, 218))))  # Model 7
hold.append(select_columns([105, 41, 44, 10, 11] + list(range(109, 150)) + list(range(177, 218))))  # Model 8
hold.append(select_columns([105, 69, 76, 70, 77] + list(range(109, 150)) + list(range(177, 218))))  # Model 9

In [5]:
# 10000 Bootstrap producing dataframes for each model 
boot_dir = Path("C:/Users/chris/OneDrive/Documentos/Coding econ 148") #Specify where the pickle documents will be saved
boot_dir.mkdir(parents=True, exist_ok=True)
for j in range(9):
    print(f"Running bootstrap for model {j + 1}...")

    # Custom dropna logic
    if j == 2:
        # Model 3: drop NAs in columns 2 and 4
        aa = hold[j].dropna(subset=[hold[j].columns[1], hold[j].columns[3]])
    elif j == 3:
        # Model 4: drop NAs in columns 2 and 6
        aa = hold[j].dropna(subset=[hold[j].columns[1], hold[j].columns[5]])
    else:
        # All other models: drop NAs in column 2 only
        aa = hold[j].dropna(subset=[hold[j].columns[1]])
    coefs = []
    successes = 0
    for i in range(10000):
        sample = aa.sample(n=len(aa), replace=True)
        y = sample.iloc[:, 0]
        X = sm.add_constant(sample.drop(sample.columns[[0]], axis=1))
        try:
            model = sm.OLS(y, X).fit()
            # Collect coefficient Series excluding constant, keep names
            coef_series = model.params.iloc[1:7]
            coefs.append(coef_series)
        except Exception as e:
            continue
        if i % 1000 == 0:
            print(f"Model {j + 1}, Iteration {i}")
    
    coefs_df = pd.DataFrame(coefs)
    coefs_df.columns = model.params.index[1:7]  # keep names from last model
    coefs_df.reset_index(drop=True, inplace=True)

    # Save as pickle
    coefs_df.to_pickle(boot_dir / f"bootstrap_df_{j + 1}.pkl")
    print(f"Saved bootstrap coefficients DataFrame for model {j + 1}.")

Running bootstrap for model 1...
Model 1, Iteration 0



KeyboardInterrupt



In [6]:
#Visualization of one of the generated bootstraps
coefs_model_5 = pd.read_pickle('bootstrap_df_5.pkl')
coefs_model_5

,temp_all,temp_all_lag,prec_all,prec_all_lag,iccode1,iccode2
0,0.022393,0.003694,-0.161681,0.048999,-0.058360,-0.205908
1,0.079980,0.017002,0.010611,-0.038418,-0.355210,-0.490695
2,0.013802,0.040866,-0.065537,0.007923,-0.208283,-0.327901
3,0.077425,0.005737,0.033499,-0.036412,-0.259383,-0.412413
4,0.097612,0.011661,0.103396,-0.077643,-0.396378,-0.529195
...,...,...,...,...,...,...
9995,0.053746,0.019273,0.014549,0.110348,-0.395497,-0.373614
9996,0.059895,0.020131,0.179957,-0.002486,-0.439697,-0.414301
9997,0.103791,0.034261,0.085373,0.187534,-0.603585,-0.601656
9998,0.036131,0.001718,0.083771,0.013015,-0.225605,-0.256732
